In [1]:
# BeatSaber Ogg Onset Detection
# 2/16/2019
# David Haas, Ian Boll, Josh Mosier, Michael Keays

import numpy as np
import matplotlib.pyplot as plt
import pickle
import glob

In [2]:
training_set_path = "../data/train120.pkl"

opened = pickle.load(open( training_set_path, "rb" ))

In [3]:
data,labels = opened

for i, ian in enumerate(zip(data,labels)):
    d,l = ian
    if l.shape[0] < d.shape[0]:
        diff = d.shape[0] - l.shape[0]
        labels[i] = np.concatenate((labels[i], np.zeros(diff, dtype=np.bool)))

training_cutoff = int(len(data) * .8)

# Concatenate each song's data into a continuous list
train_data = np.concatenate(data[:training_cutoff]).swapaxes(1, 3)
test_data = np.concatenate(data[training_cutoff:]).swapaxes(1, 3)

train_labels = np.concatenate(
    labels[:training_cutoff]).astype(np.short, copy=False)
test_labels = np.concatenate(
    labels[training_cutoff:]).astype(np.short, copy=False)


In [4]:
train_labels

array([0, 0, 0, ..., 0, 0, 0], dtype=int16)

In [5]:
train_labels.shape

(1805661,)

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout

#create model
model = Sequential()

# add model layers
model.add(Conv2D(filters=10, kernel_size=(3,7), activation='relu', input_shape=(80,15,1)))
model.add(MaxPooling2D(pool_size=(3, 1)))
model.add(Conv2D(filters=10, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 1)))
model.add(Flatten())
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.add(Dropout(0.5))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 78, 9, 10)         220       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 26, 9, 10)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 7, 10)         910       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 7, 10)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 560)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               143616    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
__________

In [7]:
model.fit(train_data, train_labels, validation_data=(test_data, test_labels), epochs=3)

Train on 1805661 samples, validate on 435406 samples
Epoch 1/3
1274944/1805661 [====================>.........] - ETA: 4:10:48 - loss: 0.2181 - acc: 0.9787

KeyboardInterrupt: 

In [ ]:
score = model.evaluate(test_data, test_labels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
song1_length = int(labels[0].shape[0]/2)
predict = model.predict(test_data[0:song1_length])
predict = [k[0] for k in predict]
actual = test_labels[0:song1_length]

In [ ]:
plt.figure(figsize=(16,6))
plt.subplot(211)
plt.plot(predict, linewidth=2.0)

plt.subplot(212)
plt.plot(actual, linewidth=2.0)
plt.show()